In [ ]:
import json
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Token ve model yükleme 
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token="token")
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, token="token", device_map="auto", torch_dtype="auto")

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto"
)

# Veriyi yükle
data_path = "/kaggle/input/kbb-test-data/kbb_qa_test.json"
with open(data_path, "r", encoding="utf-8") as f:
    all_data = json.load(f)

# Few-shot için 3 örnek seçelim 
fewshot_examples = random.sample(all_data, 3)

def build_fewshot_prompt(fewshot_examples, test_question):
    prompt = ""
    for ex in fewshot_examples:
        prompt += f"Soru: {ex['question'].strip()}\nCevap: {ex['answer'].strip()}\n\n"
    prompt += f"Soru: {test_question}\nCevap:"
    return prompt

# Test için yeni 10 soru seç
test_samples = random.sample(all_data, 10)

results = []
for idx, item in enumerate(test_samples):
    question = item["question"].strip()
    prompt = build_fewshot_prompt(fewshot_examples, question)
    
    print(f"\n[{idx+1}] Soru: {question}")
    outputs = qa_pipeline(prompt)
    model_answer = outputs[0]['generated_text'].split("Cevap:")[-1].strip()
    doctor_answer = item.get("answer", "").strip()

    print("[Doktor Cevabı]:", doctor_answer)
    print("[Model Cevabı]:", model_answer)

    results.append({
        "question": question,
        "doctor_answer": doctor_answer,
        "model_answer": model_answer
    })

# Sonuçları kaydet
with open("mistral_fewshot_zero_shot_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
